# V. Rasters and Zonal Statistics


## 4.Работа с растровыми данными на примере данных о населении (World Pop)


Экспортируем данные о населении для всей России (опционально, набор для работы уже лежит в директории)


In [ ]:
# Ссылка на данные WorldPop
url = "https://data.worldpop.org/GIS/Population/Global_2000_2020_Constrained/2020/BSGM/RUS/rus_ppp_2020_constrained.tif"

# Имя файла для сохранения
output_file = "worldpop_russia_2020.tif"

# Скачивание файла
response = requests.get(url, stream=True)

if response.status_code == 200:
    with open(output_file, "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    print(f"File saved as {output_file}")
else:
    print("Failed to download file. Status code:", response.status_code)


Открываем скачанный файл


In [ ]:
import rasterio

with rasterio.open(output_file) as dataset:
    print("CRS:", dataset.crs)
    print("Bounds:", dataset.bounds)
    print("Resolution:", dataset.res)


Обрезаем данные о численности населения по границе города (area - границы вашего района в СRS - WGS84)


In [ ]:
from rasterio.mask import mask
from shapely.geometry import mapping

# Загрузка векторных данных
# city_gdf = gpd.read_file("city_boundary.geojson")
city_geometry = [mapping(area.geometry.unary_union)]

# Вырезание только bbox для более компактного растра
with rasterio.open(output_file) as src:
    # Вычисление обрезки по bbox
    bbox = area.total_bounds  # [minx, miny, maxx, maxy]
    window = src.window(*bbox)
    
    # Читаем данные в окне
    data = src.read(window=window)
    cropped_transform = src.window_transform(window)
    profile = src.profile

    profile.update({
        "height": data.shape[1],
        "width": data.shape[2],
        "transform": cropped_transform
    })
    
    # Сохраняем уменьшенный растр
    with rasterio.open("cropped_population.tif", "w", **profile) as dst:
        dst.write(data)
    

Получаем информацию о сохраненных данных


In [ ]:
# Открытие файла
file_path = "final_population.tif"
with rasterio.open(file_path) as src:
    # Вывод основной информации о растре
    print("CRS:", src.crs)  # Система координат
    print("Bounds:", src.bounds)  # Границы растра
    print("Width, Height:", src.width, src.height)  # Размер растра
    print("Number of bands:", src.count)  # Количество слоев
    print("Data type:", src.dtypes)  # Тип данных
    print("Transform:", src.transform)  # Аффинная трансформация


Получаем информацию о размере одного пикселя


In [ ]:
with rasterio.open(file_path) as src:
    transform = src.transform  # Аффинная трансформация растра

    # Извлечение размеров пикселя
    pixel_width = transform.a  # Размер пикселя по оси X
    pixel_height = -transform.e  # Размер пикселя по оси Y (берем с минусом, так как Y направлен вниз в системе координат)

    print(f"Pixel Width: {pixel_width} units")
    print(f"Pixel Height: {pixel_height} units")


Перепроецируем растр в UTM


In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

output_file = "worldpop_reprojected.tif"

with rasterio.open(file_path) as src:
    transform, width, height = calculate_default_transform(src.crs, target_crs, src.width, src.height, *src.bounds)
    profile = src.profile
    profile.update({
        'crs': target_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(output_file, 'w', **profile) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.nearest
            )


И еще раз посмотрим на информацию о размере одного пикселя


In [ ]:
with rasterio.open(output_file) as src:
    transform = src.transform 

    # Извлечение размеров пикселя
    pixel_width = transform.a  # Размер пикселя по оси X
    pixel_height = -transform.e  # Размер пикселя по оси Y (берем с минусом, так как Y направлен вниз в системе координат)

    print(f"Pixel Width: {pixel_width} units")
    print(f"Pixel Height: {pixel_height} units")


Извлечение данных из растра


In [ ]:
with rasterio.open(file_path) as src:
    # Считываем первый слой (или единственный, если это однобандовый растр)
    data = src.read(1)  # Чтение 1-го слоя

    # Выводим статистику
    print("Min value:", np.min(data))
    print("Max value:", np.max(data))
    print("Mean value:", np.mean(data))


In [ ]:
import matplotlib.pyplot as plt

with rasterio.open(output_file) as src:
    data = src.read(1)  # Чтение 1-го слоя

    plt.figure(figsize=(10, 10))
    plt.imshow(data, cmap='viridis')  # Визуализация с использованием цветовой карты
    plt.colorbar(label="Population Density")  # Добавляем цветовую шкалу
    plt.title("Population Density (WorldPop, Russia, 2020)")
    plt.show()


## 5.Расчет плотности населения по кварталам на основе данных World Pop


Зональная статистика


In [ ]:
import rasterstats as rs

# Рассчитываем зональную статистику (сумма значений растра для каждого полигона)
stats = rs.zonal_stats(blocks_with_buildings, output_file, stats="sum", geojson_out=True)

# Преобразуем результат в GeoDataFrame
gdf_stats = gpd.GeoDataFrame.from_features(stats)

# Теперь у вас есть GeoDataFrame с дополнительной колонкой, содержащей сумму значений для каждого полигона
gdf_stats.head()


In [ ]:
gdf_stats = gdf_stats.set_crs(target_crs)

In [ ]:
gdf_stats.explore(tiles='cartodbpositron')

Вычисляем плотность населения


In [ ]:
gdf_stats['density'] = gdf_stats['sum']/(gdf_stats.geometry.area/1000000)

Смотрим на результат


In [ ]:
gdf_stats.explore(column='density')

<p style="color:#58568E; font-style:italic">The module is in progress</p>
